# Homework 2

In [41]:
import pandas as pd

import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize # сегментация русскоязычного текста на токены и предложения
import pymorphy2

In [42]:
news = pd.read_csv('articles.csv')
print(news.shape)
news.head(2)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...


In [43]:
news.iloc[2]['title']

'Форвард «Авангарда» Томаш Заборский прокомментировал игру своей команды в матче чемпионата КХЛ против «Атланта»n(4:3)n.nn«Мы провели плохой матч в Нижнем Новгороде против «Торпедо» и настраивались, что с первых же минут включимся в работу, — сказал Заборский. — У нас получилось забросить быстрый гол и задать хороший темп поединку. Мы можем играть еще лучше, но, с другой стороны, пять очков на выезде из девяти — это лучше, чем ничего».'

In [44]:
users = pd.read_csv('users_articles.csv')
users.sample(3)

,uid,articles
4783,u102831,"[5889, 6161, 7120, 7457, 6868, 511582]"
4105,u106584,"[323803, 323317, 322689, 324505, 322389, 3250]"
5750,u105590,"[294952, 290233, 293212, 294179, 290186, 1419]"


In [45]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ad_al\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

151


In [47]:
with open('stopwords.txt', encoding='utf-8') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]

In [48]:
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [49]:
# очистка текста

def clean_text(text):
    if not isinstance(text, str):
        text = str(text)
        
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub('-\s\r\n\|-\s\r\n|\r\n', '', str(text))
    
    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

In [50]:
cache = {}
morph = pymorphy2.MorphAnalyzer()

In [70]:
# Лемматизация

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''
    
    # [0]
    if not isinstance(text, str):
        text = str(text)
        
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]
    
    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru]
    
    return words_lem_without_stopwords

In [71]:
news['title'].iloc[:2].apply(lambda x: clean_text(x))

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
Name: title, dtype: object

In [64]:
%%time

from tqdm import tqdm
tqdm.pandas()

news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

100%|███████████████████████████████████████████████████████████████████████████| 27000/27000 [00:45<00:00, 595.89it/s]

CPU times: total: 42.6 s
Wall time: 45.3 s


In [67]:
news['title'].iloc[:2]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
Name: title, dtype: object

In [72]:
news['title'].iloc[:2].apply(lambda x: lemmatization(x))

0    [заместитель, председатель, правительство, рф,...
1    [матч, финал, кубок, россия, футбол, приостано...
Name: title, dtype: object

In [73]:
%%time

news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|████████████████████████████████████████████████████████████████████████████| 27000/27000 [06:01<00:00, 74.68it/s]

CPU times: total: 5min 44s
Wall time: 6min 1s


In [75]:
# формируем список наших текстов
texts = list(news['title'].values)

In [76]:
# Создаем корпус из списка с текстами
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [77]:
len(common_dictionary)

135645